<a href="https://colab.research.google.com/github/leoscarlato/math-language-project/blob/main/Projeto_Linguagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NOME da linguagem: **{mat}**

POR: Alexandre Magno e Leonardo Scarlato


**{mat}** consiste em uma linguagem de programação interpretada, com a finalidade de facilitar a escrita em português de operações matemáticas. Nesta linguagem, há palavras reservadas que representam operações que proximam a dialética matemática oral com o código. Assim, proporciando um espaço mais amigavél para estudos de ambas as ciências.

O público alvo dessa linguagem são de jovens no estágio de desenvolvimento do estudo matemático e de programação.

**{mat}** lidará com números pertencentes ao conjunto dos Racionais(Q). Contudo, na linguagem, trabalha-se com a separação entre inteiros (int) e fracionarios (float).

In [779]:
import numpy as np
import rply
import math

In [780]:
# Analisador Lexico

from rply import LexerGenerator

lg = LexerGenerator()

lg.add("NUMBER", r"\d+(.\d)?")
lg.add("PLUS", r"\+")
lg.add("MINUS", r"\-")
lg.add("MUL", r"\*")
lg.add("DIV", r"/")
lg.add("PRINT", r'PRINT')
lg.add("ROOT", r"raiz_de")
lg.add("POW", r"elevado_a")
lg.add("FACT", r"fatorial")
lg.add("PI", r"pi")
lg.add("SIN", r"seno")
lg.add("COS", r"cosseno")
lg.add("TAN", r"tangente")
lg.add("SEMICOL", r"\;")

lg.add("INT", r"int")
lg.add("FLOAT", r"float")

lg.add('ID', r'[a-zA-z][a-zA-z0-9]*')
lg.add('COMP',r'==')
lg.add('COMP',r'!=')
lg.add('COMP',r'>=')
lg.add('COMP',r'>')
lg.add('COMP',r'<=')
lg.add('COMP',r'<')
lg.add('EQUALS', r"=")

lg.ignore('\s+')

lexer = lg.build()

In [781]:
# Classes da Arvore Sintatica

from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self,decls,stmts):
        self.decls = decls
        self.stmts = stmts
    
    def accept(self, visitor):
        visitor.visit_prog(self)

class Declatations(BaseBox):
    def __init__(self,decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_declarations(self)

class Declaration(BaseBox):
    def __init__(self,id,typee):
        self.id = id
        self.typee = typee
        
    def accept(self, visitor):
        visitor.visit_declaration(self)

class Statements(BaseBox):
    def __init__(self,stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self,id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)

class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1

    def accept(self, visitor):
        visitor.visit_while(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        return visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value

class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
    pass

class Sub(BinaryOp):
    pass

class Mul(BinaryOp):
    pass

class Div(BinaryOp):
    pass

class Pow(BinaryOp):
    pass

class Root(BinaryOp):
    pass

class Fact(BinaryOp):
    pass

class Sin(BinaryOp):
    pass

class Cos(BinaryOp):
    pass

class Tan(BinaryOp):
    pass

In [782]:
# Analisador Sintatico

from rply import ParserGenerator

pg = ParserGenerator(
    # ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
    #  'PLUS', 'MINUS', 'MUL', 'DIV', 'INT', 'FLOAT', 'ID',
    #  'EQUALS','COMP','IF','ELSE', 'WHILE', 'ROOT', 
    #  'POW', 'FACT', 'PI', 'SIN', 'COS', 'TAN'
    # ],

    # precedence = [
    #     ('left', ['PLUS', 'MINUS']),
    #     ('left', ['MUL', 'DIV']),
    #     ('left', ['ROOT', 'POW', 'SIN', 'COS', 'TAN']),
    # ]

    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV', 'INT', 'FLOAT', 'ID',
     'EQUALS','COMP','IF','ELSE', 'WHILE', 'ROOT', 
     'POW', 'FACT', 'PI', 'SEMICOL'
    ],

    precedence = [
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV']),
        ('left', ['ROOT', 'POW']),
    ]
)

@pg.production('prog : declarations statements')
def prog(p):
    return Prog(p[0], p[1])

@pg.production('prog : declarations')
def prog(p):
    return Prog(p[0], None)

# --- ---

@pg.production('declarations : declaration')
def declarations(p):
    return Declatations(p[0], None)

@pg.production('declarations : declaration declarations')
def declarations(p):
    return Declatations(p[0], p[1])

@pg.production('declaration : INT ID SEMICOL')
def declaration(p):
    return Declaration(p[1].getstr(), "int")

@pg.production('declaration : FLOAT ID SEMICOL')
def declaration(p):
    return Declaration(p[1].getstr(), "float")

# --- openstatement --- #

@pg.production('statements : openstatement')
def statement_statements(p):
    return Statements(p[0],None)

@pg.production('statements : openstatement statements')
def statement_statements(p):
    return Statements(p[0],p[1])

@pg.production('openstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def expression_ifelse1(p):
    return IfElse(p[2],p[3],p[4],p[6],None)

@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE openstatement')
def expression_ifelse1(p):
    return IfElse(p[2],p[3],p[4],p[6],p[8])

@pg.production('openstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def statement_while(p):
    return While(p[2],p[3],p[4],p[6])

# --- closedstatement --- #

@pg.production('closedstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('closedstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE closedstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('closedstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr())

# @pg.production('expression : PRINT OPEN_PARENS expression CLOSE_PARENS SEMICOL')
# def expression_id(p):
#     return Id(p[0].getstr())

@pg.production('expression : NUMBER')
def expression_number(p):
    return Number(eval(p[0].getstr()))

@pg.production('expression : PI')
def expression_number(p):
    return Number(np.pi)

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
@pg.production('expression : expression ROOT expression')
@pg.production('expression : expression POW expression')
@pg.production('expression : expression FACT')
# @pg.production('expression : SIN expression')
# @pg.production('expression : COS expression')
# @pg.production('expression : TAN expression')
def expression_binop(p):
    left = p[0]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, p[2])
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, p[2])
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, p[2])
    elif p[1].gettokentype() == 'DIV':
        return Div(left, p[2])
    elif p[1].gettokentype() == 'ROOT':
        return Root(left, p[2])
    elif p[1].gettokentype() == 'POW':
        return Pow(left, p[2])
    elif p[1].gettokentype() == 'FACT':
        return Fact(left, None)
    # elif p[1].gettokentype() == 'SIN':
    #     return Sin(left, None)
    # elif p[1].gettokentype() == 'COS':
    #     return Cos(left, None)
    # elif p[1].gettokentype() == 'TAN':
    #     return Tan(left, None)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [783]:
code = """
    int x; 
    int y; 
    float z;
    int a;
    x = 2 elevado_a 5; 
    y = 16 raiz_de 4;
    z = 5 fatorial; 
    z = x + y + z;
    a = 10 + 2.2;
"""

arvore = parser.parse(lexer.lex(code))

In [784]:
class Visitor(BaseBox):
    pass

In [785]:
ST = {}
VAR = {}

class SymbolTable(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)

    def visit_declarations(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_declaration(self, d):
        ST[d.id]=d.typee
        VAR[d.id] = 0


In [786]:
arvore.accept(SymbolTable())
print(ST)

{'x': 'int', 'y': 'int', 'z': 'float', 'a': 'int'}


In [787]:
class Decorator(Visitor):
    def visit_prog(self, p):
        p.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
            d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)
    
    def visit_atrib(self, i):
        if i.id in ST:
            i.decor_type=ST[i.id]
        else:
            raise AssertionError(f'{i.id} not declared')
        i.expr.accept(self)
    
    def visit_ifelse(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
        if i.ie2!=None:
            i.ie2.accept(self)

    def visit_while(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
    
    def visit_id(self, i):
        if i.value in ST:
            i.decor_type=ST[i.value]
        else:
            raise AssertionError('id not declared')

    def visit_number(self, i):
        i.decor_type=i.value.__class__
    
    def binaryOpVisit(self, a):
        a.left.accept(self)
        a.right.accept(self)
        a.decor_type = a.left.decor_type

    def visit_add(self, a):
        self.binaryOpVisit(a)
    
    def visit_sub(self, a):
        self.binaryOpVisit(a)

    def visit_mul(self, a):
        self.binaryOpVisit(a)

    def visit_div(self, a):
        self.binaryOpVisit(a)
    
    def visit_root(self, a):
        self.binaryOpVisit(a)

    def visit_pow(self, a):
        self.binaryOpVisit(a)

    def visit_fact(self, a):
        a.left.accept(self)
        a.decor_type = a.left.decor_type


In [788]:
arvore.accept(Decorator())

In [789]:
class TypeVerifier(Visitor):

    def visit_prog(self, p):
        p.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
            d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        pass
        # if i.decor_type!=i.expr.decor_type:
        #     raise AssertionError('type error')

    def visit_ifelse(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
            raise AssertionError('type error')

    def visit_while(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
            raise AssertionError('type error')

In [790]:
arvore.accept(TypeVerifier())

In [791]:
class Eval(Visitor):
    def visit_prog(self, p):
        p.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
            d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, d):
        # d.expr = d.expr.accept(self)
        # d.expr = int(d.expr) if ST[d.id] == "int" else float(d.expr)
        # VAR[d.id] = d.expr
        num = d.expr.accept(self)
        VAR[d.id] = int(num) if ST[d.id] == "int" else float(num)

    def visit_id(self, d):
        return VAR[d.value]

    def visit_number(self, number):
        return number.value

    def visit_add(self, d):
        return d.left.accept(self) + d.right.accept(self)
    
    def visit_sub(self, d):
        return d.left.accept(self) - d.right.accept(self)
    
    def visit_mul(self, d):
        return d.left.accept(self) * d.right.accept(self)

    def visit_div(self, d):
        return d.left.accept(self) / d.right.accept(self)
    
    def visit_root(self, d):
        return d.left.accept(self) ** (1/d.right.accept(self))
    
    def visit_pow(self, d):
        return d.left.accept(self) ** d.right.accept(self)
    
    def visit_fact(self, d):
        return math.factorial(d.left.accept(self))

In [792]:
arvore.accept(Eval())

In [793]:
class Print(Visitor):
    def visit_prog(self, p):
        p.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
            d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, d):
        # print(f'{d.id} = {VAR[d.id]}')
        print(f'{d.decor_type} {d.id} = {d.expr} ')

In [794]:
arvore.accept(Print())

int x = <__main__.Pow object at 0x000001BF1A337E10> 
int y = <__main__.Root object at 0x000001BF1A336250> 
float z = <__main__.Fact object at 0x000001BF1A337D50> 
float z = <__main__.Add object at 0x000001BF19AC3A90> 
int a = <__main__.Add object at 0x000001BF19AC35D0> 


In [795]:
for k in VAR.keys():
    print(f'{ST[k]} {k} = {VAR[k]}')


int x = 32
int y = 2
float z = 154.0
int a = 12
